## Word Embeddings

Librerías necesarias para el Lab.

In [25]:
import os
import gensim
from gensim.models import KeyedVectors, FastText
import numpy as np
import pandas as pd
import glob
import warnings
warnings.filterwarnings("ignore")

### Read Data

In [2]:
handlers = ['sanchezcastejon', 'pablocasado_',
     'Pablo_Iglesias_', 'Santi_ABASCAL', 'Albert_Rivera']

In [5]:
data = {}
for handler in handlers:
    data[handler] = pd.read_csv('data/' + handler + '.csv')

In [10]:
df = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "data/*.csv"))))

In [11]:
df.head()

,tweet,created_at,id
0,Estos 3 minutos de @EspejoPublico a @PedroRuiz...,Tue Dec 03 18:10:08 +0000 2019,1.201927e+18
1,😂😂😂 https://t.co/C2dJ2TjGwo,Sat Nov 30 17:39:22 +0000 2019,1.200832e+18
2,Lo advertimos antes del 10N. \nEl PSOE de Sánc...,Tue Nov 26 11:17:34 +0000 2019,1.199286e+18
3,"¡Sois muy grandes, campeones! Habéis vuelto a ...",Sun Nov 24 20:37:25 +0000 2019,1.198702e+18
4,Decía el presidente Kennedy que lo inteligente...,Sun Nov 24 10:43:17 +0000 2019,1.198553e+18


### Word2Vec basic model

Word2Vec input format: list of tokens in documents 

In [60]:
dataset = list(df['tweet'])
dataset = [sent.split(' ') for sent in dataset]

In [17]:
model = gensim.models.Word2Vec(
        dataset,
        size=2,
        sg=1, #Skip-Gram
        min_count=0)

In [18]:
model.wv.get_vector('economía')

array([-2.5368118 , -0.86686504], dtype=float32)

In [22]:
w1 = ["economía"]
model.wv.most_similar (positive=w1,topn=5)

[('https://t.co/CMCVS5VoaI', 1.0),
 ('bien', 1.0),
 ('presentadas', 1.0),
 ('https://t.co/EbNzjF4E4E', 1.0),
 ('https://t.co/wu0KiBW5Sr', 1.0)]

In [24]:
w1 = ["educación"]
model.wv.most_similar (positive=w1,topn=5)

[('lacra.\nMi', 1.0000001192092896),
 ('https://t.co/ntZTP3bkUH', 1.0),
 ('populistas', 1.0),
 ('@PPopular', 1.0),
 ('https://t.co/r3DV4mOlha', 1.0)]

### FastText

In [64]:
model = FastText(size=2, min_count=0)  # instantiate
model.build_vocab(sentences=dataset)
model.train(sentences=dataset, total_examples=len(dataset), epochs=5)

In [65]:
model.wv.get_vector('economía')

array([-4.99747   ,  0.05966096], dtype=float32)

In [66]:
w1 = ["economía"]
model.wv.most_similar (positive=w1,topn=5)

[('Inmoral.', 1.0),
 ('historia!!', 1.0),
 ('#transiciónecológica,', 1.0),
 ('unanimidad', 1.0),
 ('🏆Campeón', 1.0)]

## Preprocessing

In [30]:
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
import nltk
import spacy
import unicodedata

In [31]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ermartin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

### Standarization

In [34]:
dataset = list(df['tweet'])
dataset[0]

'Estos 3 minutos de @EspejoPublico a @PedroRuiz_Photo contando las historias y los sentimientos que hay detrás de cada foto son una joya. No os lo perdáis. ¡Gracias, Pete! Yo también he perdido a un fotógrafo pero he ganado un gran amigo 😉📸 https://t.co/OU9eH1NKEw'

In [33]:
data_lwr = []
for sent in dataset:
    data_lwr.append(sent.lower())
data_lwr[0]

'estos 3 minutos de @espejopublico a @pedroruiz_photo contando las historias y los sentimientos que hay detrás de cada foto son una joya. no os lo perdáis. ¡gracias, pete! yo también he perdido a un fotógrafo pero he ganado un gran amigo 😉📸 https://t.co/ou9eh1nkew'

In [35]:
data_std = []
for sent in data_lwr:
    data_std.append(unicodedata.normalize('NFKD', sent).encode('ASCII', 'ignore').decode('utf-8'))
data_std[0]

'estos 3 minutos de @espejopublico a @pedroruiz_photo contando las historias y los sentimientos que hay detras de cada foto son una joya. no os lo perdais. gracias, pete! yo tambien he perdido a un fotografo pero he ganado un gran amigo  https://t.co/ou9eh1nkew'

### Tokenize

In [36]:
data_tkn = []
for sent in data_std:
    data_tkn.append(wordpunct_tokenize(sent))
data_tkn[0]

['estos',
 '3',
 'minutos',
 'de',
 '@',
 'espejopublico',
 'a',
 '@',
 'pedroruiz_photo',
 'contando',
 'las',
 'historias',
 'y',
 'los',
 'sentimientos',
 'que',
 'hay',
 'detras',
 'de',
 'cada',
 'foto',
 'son',
 'una',
 'joya',
 '.',
 'no',
 'os',
 'lo',
 'perdais',
 '.',
 'gracias',
 ',',
 'pete',
 '!',
 'yo',
 'tambien',
 'he',
 'perdido',
 'a',
 'un',
 'fotografo',
 'pero',
 'he',
 'ganado',
 'un',
 'gran',
 'amigo',
 'https',
 '://',
 't',
 '.',
 'co',
 '/',
 'ou9eh1nkew']

### Punctuation

In [37]:
data_pnkt = []
for sent in data_tkn:
    data_pnkt.append([word for word in sent if word.isalpha()])
data_pnkt[0]

['estos',
 'minutos',
 'de',
 'espejopublico',
 'a',
 'contando',
 'las',
 'historias',
 'y',
 'los',
 'sentimientos',
 'que',
 'hay',
 'detras',
 'de',
 'cada',
 'foto',
 'son',
 'una',
 'joya',
 'no',
 'os',
 'lo',
 'perdais',
 'gracias',
 'pete',
 'yo',
 'tambien',
 'he',
 'perdido',
 'a',
 'un',
 'fotografo',
 'pero',
 'he',
 'ganado',
 'un',
 'gran',
 'amigo',
 'https',
 't',
 'co']

### Remove Stopwords

In [44]:
sw = stopwords.words('spanish')
sw[:10]

['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se']

In [45]:
data_stp = []
for sent in data_pnkt:
    data_stp.append([word for word in sent if not word in sw])
data_stp[0]

['minutos',
 'espejopublico',
 'contando',
 'historias',
 'sentimientos',
 'detras',
 'cada',
 'foto',
 'joya',
 'perdais',
 'gracias',
 'pete',
 'tambien',
 'perdido',
 'fotografo',
 'ganado',
 'gran',
 'amigo',
 'https',
 't',
 'co']

### Lemma

In [46]:
lemm = spacy.blank('es')
data_lemm = []
for sent in data_stp:
    data_lemm.append([tokens.lemma_ for tokens in lemm(' '.join(word for word in sent))])
data_lemm[0]

['minuto',
 'espejopublico',
 'contar',
 'historia',
 'sentimiento',
 'detras',
 'cada',
 'foto',
 'joya',
 'perdais',
 'gracia',
 'petar',
 'tambien',
 'perder',
 'fotografo',
 'ganar',
 'gran',
 'amigar',
 'https',
 't',
 'co']

### Stemming

In [48]:
stemmer = SnowballStemmer('spanish')
data_stem = []
for sent in data_stp:
    data_stem.append([stemmer.stem(word) for word in sent])
data_stem[0]

['minut',
 'espejopubl',
 'cont',
 'histori',
 'sentimient',
 'detr',
 'cad',
 'fot',
 'joy',
 'perdais',
 'graci',
 'pet',
 'tambi',
 'perd',
 'fotograf',
 'gan',
 'gran',
 'amig',
 'https',
 't',
 'co']

### Word2Vec + Preprocessing + Hiperparameters

In [71]:
model = gensim.models.Word2Vec(
        data_lemm,
        size=300,
        sg=0, #Skip-Gram
        min_count=0,
        iter=50)

In [72]:
model.wv.get_vector('economia')

array([ 0.22485635,  1.9060785 , -0.7529961 , -0.73270696, -1.337491  ,
        1.2403338 ,  0.24137034,  0.7897106 ,  0.83259606, -0.7240762 ,
       -0.5598111 ,  2.3932803 ,  0.6885952 , -1.029048  ,  0.20227851,
        0.32435822,  0.38391966, -0.9660038 ,  0.10311033, -0.08994564,
        0.8718674 , -0.60313493,  0.5347903 , -0.07227661,  0.60987353,
        0.0050941 , -0.6552838 , -0.89252836,  0.26420036, -0.63748   ,
       -1.3078681 , -0.25982788, -0.08865281, -1.6938784 , -0.28293437,
        1.3580698 , -0.0226552 , -0.476426  ,  0.8719193 ,  0.41868076,
        0.20205292,  0.9243562 , -0.94761443, -1.3627263 , -0.00395201,
        0.5220138 ,  1.0406401 ,  0.47904032, -0.19518796,  0.9000996 ,
       -0.63928133,  0.09340114, -0.56967634,  0.12233819, -0.6296528 ,
        0.07959235,  0.9433808 ,  0.87400055,  0.455717  , -0.2123958 ,
       -0.8102293 , -0.35573125,  0.74859947, -0.7109837 ,  0.35554492,
       -0.5320631 ,  0.2712223 ,  0.5050528 , -1.2983769 ,  0.63

In [73]:
w1 = ["economia"]
model.wv.most_similar(positive=w1,topn=5)

[('economico', 0.6003372073173523),
 ('competitividad', 0.5598915815353394),
 ('pib', 0.5515145063400269),
 ('enfriamiento', 0.551472544670105),
 ('ralentizacion', 0.543662428855896)]

## Pretrained Model

In [76]:
model_pretrained = KeyedVectors.load_word2vec_format('models/wiki.es.vec')

In [81]:
model.build_vocab([list(model_pretrained.vocab.keys())], update=True)
model.intersect_word2vec_format('models/wiki.es.vec', lockf=1.0)

In [82]:
w1 = ["economia"]
model.wv.most_similar(positive=w1,topn=5)

[('oeconomia', 0.8983333110809326),
 ('economias', 0.8931393623352051),
 ('/economia', 0.8916008472442627),
 ('//economia', 0.8205404877662659),
 ('macroeconomia', 0.8028700947761536)]

In [83]:
w1 = ["economia"]
model.wv.most_similar(positive=w1,topn=15)

[('oeconomia', 0.8983333110809326),
 ('economias', 0.8931393623352051),
 ('/economia', 0.8916008472442627),
 ('//economia', 0.8205404877662659),
 ('macroeconomia', 0.8028700947761536),
 ('economi', 0.797034740447998),
 ('microeconomia', 0.7822957634925842),
 ('economico', 0.7618166208267212),
 ('economica', 0.7474629282951355),
 ('econom', 0.7408113479614258),
 ('ekonomia', 0.7335286140441895),
 ('americaeconomia', 0.7213191390037537),
 ('economice', 0.7168940305709839),
 ('/economia/', 0.7121700048446655),
 ('economică', 0.7101243138313293)]